In [1]:
import pandas as pd
import numpy as np

In [24]:
#load data
bank_info = pd.read_csv('bank-additional-full.csv', delimiter=";")
bank_info.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [23]:
age = bank_info.age
duration = bank_info.duration
call_time = bank_info.campaign

In [15]:
j = bank_info.job
job = pd.get_dummies(j)

In [5]:
pd.options.mode.chained_assignment = None

In [16]:
m = bank_info.marital
m[(m == 'divorced')|(m == 'married')].loc[:]= 'married_current_or_before'
m[(m == 'single') | (m == 'unknown')].loc[:] = 'not_married'
marital = pd.get_dummies(m)

In [17]:
e = bank_info.education
e[(e == 'illiterate')|(e == 'basic.4y')|(e == 'basic.6y')|(e == 'basic.9y')]= 'high_school_below'
e[(e == 'university.degree')|(e == 'professional.course')|(e == 'unknown')] = 'high_school_above'
education = pd.get_dummies(e)

In [18]:
contact = pd.get_dummies(bank_info.contact)

In [19]:
ne = bank_info['nr.employed']
ne[(ne == 4963.6)|(ne == 4991.6)|(ne == 5008.7)|(ne == 5017.5)|(ne == 5023.5)|(ne == 5076.2)] = '5099_below'
ne[(ne == 5176.3)|(ne == 5191.0)|(ne == 5195.8)|(ne == 5228.1)] = '5100_above'
number_employed = pd.get_dummies(ne)

In [20]:
month = pd.get_dummies(bank_info.month)

In [27]:
part = pd.concat([age, job, marital, education, contact, month, call_time, number_employed], axis=1)
part.head()

,age,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,...,jun,mar,may,nov,oct,sep,campaign,5099.1,5099_below,5100_above
0,56,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1,57,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,1
2,37,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,1
3,40,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
4,56,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,1
